# Projet 2 - Fashion Trend Intelligence | Segmentation vestimentaire avec IA

Ce notebook permet d’évaluer la faisabilité technique du modèle SegFormer-clothes, afin de déterminer s’il est capable d’identifier et d’isoler avec précision chaque pièce vestimentaire présente dans une image.


## 1. Installation du projet et de son environnement

Afin d’utiliser correctement ce notebook, vérifiez que vous disposez du bon environnement pour l’exécuter.


### 1.1 Installation de Python

Pour ce projet, il est nécessaire d’avoir **au minimum Python 3.8**.  
Si ce n’est pas déjà le cas, vous pouvez vous référer à [la documentation officielle](https://www.python.org/downloads/).

Vérifiez votre version de Python :

```bash
python --version
```

### 1.2 Installation de `uv`

`uv` est un gestionnaire de projets Python permettant d’installer et d’organiser les dépendances plus rapidement et plus simplement que les outils traditionnels (`pip`, `virtualenv`, etc.).

Pour installer `uv`, veuillez suivre [la documentation officielle](https://docs.astral.sh/uv/getting-started/installation/#standalone-installer)

Vérifiez l’installation :

```bash
uv --version
```

### 1.3 Création du projet

Créez un nouveau projet Python avec `uv` :

```bash
uv init nom_du_projet
cd nom_du_projet
```

La structure de base du projet est alors générée automatiquement.


### 1.4 Création et activation de l’environnement virtuel

Créez l’environnement virtuel :

```bash
uv venv
```

Activez-le selon votre système :

* **Linux / macOS**

```bash
source .venv/bin/activate
```

* **Windows (PowerShell)**

```powershell
.venv\Scripts\Activate.ps1
```

Voir la documentation officielle :
[https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment](https://docs.astral.sh/uv/pip/environments/#creating-a-virtual-environment)



### 1.5 Installation des dépendances

> ❗ Assurez-vous que l’environnement virtuel est **activé** avant d’installer les dépendances.

Installez les bibliothèques nécessaires au projet :

```bash
uv add ipykernel jupyterlab requests pillow matplotlib numpy tqdm python-dotenv
```

Ces dépendances sont automatiquement enregistrées dans le fichier `pyproject.toml`.

Vous pouvez visualiser l’ensemble des dépendances installées avec la commande suivante :

```bash
uv tree
```

### 1.6 Création du fichier `.env`

Créez un fichier nommé `.env` à la racine du projet.

Copiez-y le contenu suivant et remplacez les valeurs si nécessaire :

```env
# Token d'authentification à l'API Hugging Face
HF_API_TOKEN=VOTRE_TOKEN_HUGGING_FACE_ICI

# Chemins du dataset
DATASET_IMAGES_DIR=content/images_a_segmenter
DATASET_ANNOTATIONS_DIR=content/annotations
```

⚠️ **Important** :

* Ne partagez jamais votre fichier `.env`
* Ajoutez-le à votre `.gitignore`

```gitignore
.env
```

### 1.7 Création d’un token Hugging Face

1. Créez un compte sur [https://huggingface.co/](https://huggingface.co/)
2. Allez dans **Profile → Settings → Access Tokens**
3. Créez un nouveau token (rôle **read** suffisant)
4. Copiez le token dans la variable `HF_API_TOKEN` du fichier `.env`


### 1.8 Préparation des jeux de données

Récupérez les jeux de données (images et annotations) et placez-les dans les dossiers suivants :

```
content/
├── images_a_segmenter/
│   ├── image_0.jpg
│   ├── image_1.jpg
│   └── ...
└── annotations/
    ├── mask_0.json
    ├── mask_1.json
    └── ...
```

## 2. Importation des Bibliothèques

Commençons par importer les bibliothèques Python nécessaires. Nous aurons besoin de :
- **`os`** : Interaction avec le système de fichiers (navigation dans les répertoires, listage des fichiers images).
- **`requests`** : Envoi de requêtes HTTP vers l'API pour la segmentation d'images.
- **`PIL (Pillow)`** : Manipulation et traitement des images (ouverture, redimensionnement, conversion).
- **`matplotlib.pyplot`** : Visualisation des images originales et des masques de segmentation.
- **`matplotlib.patches`** : Création d'éléments graphiques personnalisés pour les légendes des visualisations.
- **`numpy`** : Manipulation efficace des tableaux numériques représentant les pixels des images.
- **`tqdm.notebook`** : Affichage d'une barre de progression interactive dans les notebooks Jupyter (utile lors du traitement par lot).
- **`base64`** : Encodage/décodage en Base64 des images et masques échangés avec l'API.
- **`io`** : Gestion des flux de données en mémoire pour la manipulation des images sans fichiers temporaires.
- **`python-dotenv`** : Chargement sécurisé des variables d'environnement (comme les clés API) depuis un fichier `.env`.
- **`time`** : Gestion des délais entre les appels API pour respecter les limites de taux (rate limiting).
- **`re`** : Traitement d'expressions régulières (parsing de réponses, validation de formats).

In [1]:
import os
import requests
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
from tqdm.notebook import tqdm
import base64
import io
from dotenv import load_dotenv
import time
import re

## 3. Chargement de la Configuration

In [5]:
# Charger les variables d'environnement depuis le fichier .env
load_dotenv(override=True)

# Récupération des variables d'environnement
image_dir = os.getenv('DATASET_IMAGES_DIR')
annotations_dir = os.getenv('DATASET_ANNOTATIONS_DIR')
api_token = os.getenv('HF_API_TOKEN')
max_images = 50  # Nombre maximum d'images à traiter

# Vérification de la présence des variables
if not image_dir or not api_token or not annotations_dir:
    raise ValueError("Variables d'environnement manquantes dans le fichier .env")

# Vérification du token
if api_token == "VOTRE_TOKEN_HUGGING_FACE_ICI":
    raise ValueError("Vous devez remplacer 'VOTRE_TOKEN_HUGGING_FACE_ICI' par votre token API personnel dans le fichier .env")

# Création des dossiers s'ils n'existent pas
for directory, name in [(image_dir, "images"), (annotations_dir, "annotations")]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"✓ Dossier '{directory}' créé pour les {name}.")
    else:
        print(f"✓ Dossier '{directory}' existant.")

✓ Dossier 'content/images_a_segmenter' existant.
✓ Dossier 'content/annotations' existant.


## 3. Configuration de l'API Hugging Face

### Présentation du modèle
Nous utilisons le modèle [**SegFormer-B3-Clothes**](https://huggingface.co/sayeed99/segformer_b3_clothes) qui est spécialisé dans la segmentation sémantique de vêtements. Ce modèle peut détecter 18 classes différentes :
- Éléments vestimentaires : chapeau, haut, jupe, pantalon, robe, ceinture, chaussures, sac, écharpe
- Parties du corps : cheveux, visage, bras, jambes
- Accessoires : lunettes de soleil
- Arrière-plan

### Configuration de l'endpoint API
L'API Hugging Face Inference Router permet d'interroger le modèle sans avoir à le déployer localement. Voici les éléments de configuration :

**URL de l'API** : `https://router.huggingface.co/hf-inference/models/sayeed99/segformer_b3_clothes`

**Headers requis** :
- `Authorization: Bearer YOUR_TOKEN` : Authentification avec votre token personnel
- `Content-Type: image/jpeg` ou `image/png` : Type MIME de l'image envoyée

**Format de réponse** : L'API retourne un JSON contenant une liste de masques encodés en base64, un par classe détectée.

**Sécurité** : Ne jamais hardcoder votre token dans le notebook. Elle doit mis dans la variable d'environnement `HF_API_TOKEN'` le fichier `.env`

### Bonnes pratiques d'utilisation
1. **Rate Limiting** : Espacer les requêtes de 1-2 secondes pour éviter les erreurs 429 (Too Many Requests)
2. **Timeout** : Définir un timeout de 30 secondes minimum (le modèle peut être lent)
3. **Gestion d'erreurs** : Prévoir les cas suivants :
   - 401 : Token invalide
   - 429 : Trop de requêtes
   - 500 : Erreur serveur (réessayer après quelques secondes)
   - 503 : Modèle en cours de chargement (attendre et réessayer)

In [ ]:
API_URL = "https://router.huggingface.co/hf-inference/models/sayeed99/segformer_b3_clothes"
headers = {
    "Authorization": f"Bearer {api_token}"
    # Le "Content-Type" sera ajouté dynamiquement lors de l'envoi de l'image
}


def request_cloth_segmentation(single_image_path):
    """
    Requête à l'API Hugging Face.

    Args:
        single_image_path (str): Chemin vers l'image à segmenter

    Returns:
        dict: Résultats de la segmentation (JSON de l'API)
        
    Raises:
        FileNotFoundError: Si le fichier image n'existe pas
        ValueError: Si le format d'image n'est pas supporté
        requests.exceptions.RequestException: Si la requête API échoue
    """
    
    # On vérifie que le fichier existe
    if not os.path.exists(single_image_path):
        raise FileNotFoundError(f"L'image '{single_image_path}' n'existe pas")
    
    try:
        # On ouvre l'image pour vérifier le format (une seule fois)
        with Image.open(single_image_path) as img:
            image_format = img.format
            
            # On vérifie que le format est supporté
            if image_format not in ['JPEG', 'PNG']:
                raise ValueError(
                    f"Format '{image_format}' non supporté. "
                    f"Formats acceptés: JPEG, PNG"
                )
    
    except Exception as e:
        # Erreur lors de l'ouverture de l'image (fichier corrompu, etc.)
        raise ValueError(f"Impossible d'ouvrir l'image: {e}")
    
    # Lire le contenu binaire de l'image
    try:
        with open(single_image_path, 'rb') as fichier:
            image_data = fichier.read()
    except IOError as e:
        raise IOError(f"Erreur lors de la lecture du fichier: {e}")
    
    # On configure les headers avec le Content-Type
    headers["Content-Type"] = f"image/{image_format.lower()}"
    
    # On fait la requête POST à l'API
    try:
        response = requests.post(
            API_URL,
            headers=headers,
            data=image_data,
            timeout=30  # Timeout de 30 secondes
        )
        print(f"Réponse reçue pour {os.path.basename(single_image_path)}")

        # Si status >= 400 alors il y aura une exception
        response.raise_for_status()
        
    except requests.exceptions.Timeout:
        raise requests.exceptions.Timeout(
            f"Timeout lors de la requête API pour '{single_image_path}'"
        )
    except requests.exceptions.ConnectionError:
        raise requests.exceptions.ConnectionError(
            "Impossible de se connecter à l'API Hugging Face"
        )
    except requests.exceptions.HTTPError as e:
        # Erreur HTTP (4xx, 5xx)
        error_detail = ""
        try:
            error_detail = response.json().get('error', '')
        except:
            error_detail = response.text[:200]  # Premiers 200 caractères
        
        raise requests.exceptions.HTTPError(
            f"Erreur API (status {response.status_code}): {error_detail}"
        )
    
    # 6. Parser la réponse JSON
    try:
        results = response.json()
        return results
    except ValueError as e:
        raise ValueError(f"Réponse API invalide (pas du JSON): {e}")